# association
co_visitaion_matrixにaidごとの出現数を考慮する

In [1]:
import os
import sys
import gc
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import random
from collections import defaultdict

import pandas as pd
import cudf

In [2]:
SEED = 42
random.seed(SEED)

In [3]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

In [4]:
weeks = ["week4"]

for week in weeks:
    print(week)
        
    # データ読み込み
    files = [
        "train_sessions_week1.parquet",
        "test_sessions_week1.parquet",
        "train_sessions_week2.parquet",
        "test_sessions_week2.parquet",
        "train_sessions_week3.parquet",
        "test_sessions_week3.parquet",
        "train_sessions_week4.parquet",
        "test_sessions_week4.parquet",
        "test_sessions.parquet"
    ]

    if week is not None:
        files.remove(f"test_sessions_{week}.parquet")
        pair_df = pd.read_parquet(PREP_DIR + f"co_visitation_matrix_{week}.parquet")
    else:
        pair_df = pd.read_parquet(PREP_DIR + "co_visitation_matrix.parquet")
    pair_df.columns = ["aid_x", "aid_y", "pair_cnt", "org_rank"]
    
    print(files)
    dfs = []
    for file in files:
        dfs.append(pd.read_parquet(PREP_DIR + file))
    sessions = pd.concat(dfs)
    
    aid_cnt = sessions.groupby("aid")["ts"].count().reset_index()
    aid_cnt.columns = ["aid", "single_cnt"]

    total_cnt = len(sessions)
    
    pair_df = pair_df.merge(aid_cnt, left_on="aid_x", right_on="aid", how="left").drop(columns=["aid"])
    pair_df = pair_df.merge(aid_cnt, left_on="aid_y", right_on="aid", how="left").drop(columns=["aid"])

    pair_df["x_support"] = pair_df["single_cnt_x"] / total_cnt
    pair_df["y_support"] = pair_df["single_cnt_y"] / total_cnt
    pair_df["xy_support"] = pair_df["pair_cnt"] / total_cnt
    pair_df["lift"] = pair_df["xy_support"] / (pair_df["x_support"] * pair_df["y_support"])

    pair_df = pair_df.sort_values(["aid_x", "lift"], ascending=(True, False), ignore_index=True)
    pair_df["rank"] = pair_df.groupby(["aid_x"]).cumcount().astype("int32")
    
    if week is not None:
        pair_df[["aid_x", "aid_y", "lift", "rank"]].to_parquet(PREP_DIR + f"association_{week}.parquet")
    else:
        pair_df[["aid_x", "aid_y", "lift", "rank"]].to_parquet(PREP_DIR + f"association.parquet")

week4
['train_sessions_week1.parquet', 'test_sessions_week1.parquet', 'train_sessions_week2.parquet', 'test_sessions_week2.parquet', 'train_sessions_week3.parquet', 'test_sessions_week3.parquet', 'train_sessions_week4.parquet', 'test_sessions.parquet']


In [5]:
pair_df

,aid_x,aid_y,pair_cnt,org_rank,single_cnt_x,single_cnt_y,x_support,y_support,xy_support,lift,rank
0,0,968176,3,71,45,5,2.322031e-07,2.580035e-08,1.548021e-08,2.583945e+06,0
1,0,698599,5,26,45,9,2.322031e-07,4.644062e-08,2.580035e-08,2.392541e+06,1
2,0,369030,3,57,45,6,2.322031e-07,3.096042e-08,1.548021e-08,2.153287e+06,2
3,0,740714,4,43,45,10,2.322031e-07,5.160069e-08,2.064028e-08,1.722630e+06,3
4,0,1300298,8,11,45,27,2.322031e-07,1.393219e-07,4.128055e-08,1.276022e+06,4
...,...,...,...,...,...,...,...,...,...,...,...
339301373,1855602,1819342,3,31,19,6564,9.804132e-08,3.387070e-05,1.548021e-08,4.661692e+03,27
339301374,1855602,1517680,5,7,19,11927,9.804132e-08,6.154415e-05,2.580035e-08,4.275921e+03,28
339301375,1855602,523174,5,6,19,15482,9.804132e-08,7.988819e-05,2.580035e-08,3.294077e+03,29
339301376,1855602,479970,3,22,19,27632,9.804132e-08,1.425830e-04,1.548021e-08,1.107388e+03,30
